### **imports**

In [1]:
import pandas as pd
import numpy as np
from astropy.io import ascii
from astropy.table import QTable, Table, Column
from astropy import units as u
import string
import upsilon

-------------------
| pyFFTW detected |
-------------------


### **making a table of all of the stars in the txt file**

In [2]:
column1 = []
column2 = []
column3 = []
column4 = []

with open('rrlyr_vsx_clean.txt', 'r') as file:
    for line in file:
        empty = []
        components = line.split(' ', 3)
    
        column1.append(components[0])
        column2.append(float(components[1]))
        column3.append(float(components[2]))
        column4.append(components[3].strip())

# column2 = column2 * u.deg
# column3 = column3 * u.deg

final_table = QTable([column1, column2, column3, column4], names = ("Star Type", "RA", "Dec", "Other Info"))

In [3]:
final_table

Star Type,RA,Dec,Other Info
str24,float64,float64,str30
RRAB,0.01307,-2.43007,PS1-3PI J000003.12-022548.2
RRAB,0.01521,35.36286,GM And
RRAB,0.01669,18.40698,CSS_J000004.0+182425
RRAB,0.02771,56.65331,V1023 Cas
RRAB,0.03171,34.67397,SERIV 27
RRAB,0.03901,5.58986,CSS_J000009.3+053523
RRAB,0.04274,-2.51629,PS1-3PI J000010.25-023058.6
RRAB,0.04331,-21.92086,CSS 120828:000010-215515
RRAB,0.04782,62.71524,Gaia DR2 430046902041301376


### **importing the asassn client**

In [4]:
from pyasassn.client import SkyPatrolClient	
client = SkyPatrolClient()

Welcome to ASAS-SN Skypatrol!

Current Deployment Version: 0.6.17 (26 JAN 2024)
Please upgrade your client if not up to date.



### **downloading a csv file with the data from the star at a specified index**

In [5]:
folder_path = "/Users/zoesurles/Desktop/variable_star_classification/downloaded_lightcurves/"

In [6]:
def classification_model(path, ra, dec, vsx_id, asassn_id):
  # Load a classification model.
  # (takes time, do only once if processing multiple lcs)
  rf_model = upsilon.load_rf_model()


  ############# Read input lc data #############
  # Read the input lightcurve file
  f = open(path, 'r')
  data = np.loadtxt(f, dtype={'names': ('JD', 'mag', 'err'),'formats': ('float64', 'float64', 'float64')})
  f.close()

  # Split it into three arrays
  date=data['JD']
  mag=data['mag']
  err=data['err']

  # Sigma-clip the input lightcurve
  date, mag, err = upsilon.utils.sigma_clipping(date, mag, err,  threshold=3, iteration=1)

  ############# Extract lightcurve features #############
  e_features = upsilon.ExtractFeatures(date, mag, err)
  e_features.run()
  features = e_features.get_features()

  ############# Classify #############
  # Classify the light curve
  label, probability, flag = upsilon.predict(rf_model, features)

  # Print results

  output_file = open(path.replace('.txt', '_classification.txt'), 'w')
  output_file.write("\n\n######### Classification results #########\n")
  output_file.write("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)\n")
  output_file.write("class = " + str(label) + "\n")
  output_file.write("class_probability = " + str(probability) + "\n")
  output_file.write("position = " + str(ra) + " " + str(dec) + "\n")
  output_file.write("vsx_id = " + str(vsx_id) + "\n")
  output_file.write("asassn_id = " + str(asassn_id) + "\n")
  output_file.close()

  print("\n\n######### Classification results #########\n")
  print("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)")
  print("class = " + str(label) )
  print("class_probability = " + str(probability) )
  print("\n\n######### Lightcurve features #########\n")

  for feature in features:
    print( str(feature) + " = " + str(features[feature]) )
  #  print feature," = ",features[feature]

In [11]:
num_iterations = 10
empty_list = []
for i in range(5, 10):
    if i == 8:
        continue
    search = client.cone_search(ra_deg = final_table["RA"][i], dec_deg = final_table["Dec"][i], radius=0.0003, catalog='aavsovsx', download=True)

    savecsv = search.save(save_dir = folder_path, file_format="csv")[1]
    empty_list.append(savecsv)

    new_file = pd.read_csv(savecsv, skiprows=1)

    g_filter = new_file[new_file['phot_filter'] == 'g']
    web_format = g_filter[["jd", "mag", "mag_err"]]
    web_format.to_csv(savecsv.replace(".csv", ".txt"), sep='\t', index=False, header = False)

    classification_model(savecsv.replace(".csv", ".txt"), final_table["RA"][i], final_table["Dec"][i], final_table["Other Info"][i], new_file["asas_sn_id"][0])

Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 1.0


######### Lightcurve features #########

amplitude = 0.2983202321187974
cusum = 0.17799895701815555
eta = 2.2729342085018343
hl_amp_ratio = 0.44285351655523775
kurtosis = -0.3816686813179335
n_points = 961
period = 0.5759936800099591
period_SNR = 178.18256606351062
period_log10FAP = -103.07279697227928
period_uncertainty = 0.0001196497099851257
phase_cusum = 0.23441219081288817
phase_eta = 0.8182009227899499
phi21 = 0.7194378560183168
phi31 = 1.7335944003732617
quartile31 = 0.41417612650111835
r21 = 0.4207716724920455
r31 = 0.22984094482928333
shapiro_w = 0.9670776724815369
skewness = -0.4354867264363516
slope_per10 = -0.024314310257161277
slope_per90 = 0.024630556667586373
stetson_k = 0.6884985297621655
weighted_mean = 16.23619334938714
weighted_std = 0.27471906407814256
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 1 (0 - classification success, 1 - suspicious period)
class = CEPH_F
class_probability = 0.48


######### Lightcurve features #########

amplitude = 0.5674361762625245
cusum = 0.17192562694446986
eta = 1.3270652816380162
hl_amp_ratio = 0.6344677280552167
kurtosis = -0.4173344252831428
n_points = 873
period = 29.600766106407185
period_SNR = 119.31542776436908
period_log10FAP = -56.95415684735496
period_uncertainty = 0.31493613000596987
phase_cusum = 0.2149105897955252
phase_eta = 1.561787358414874
phi21 = -2.936030166098245
phi31 = -2.760858772364091
quartile31 = 1.0560889024042162
r21 = 0.3745373123456555
r31 = 0.1778819663819481
shapiro_w = 0.9556871652603149
skewness = -0.569874295430106
slope_per10 = -0.42046349202090366
slope_per90 = 0.31016119751359744
stetson_k = 0.13132276344462493
weighted_mean = 17.93311056395681
weighted_std = 0.6070411482891993
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 0.9


######### Lightcurve features #########

amplitude = 0.3559657322203219
cusum = 0.10753393695918277
eta = 1.4697394307836915
hl_amp_ratio = 0.6023852422999114
kurtosis = -0.7095748308524703
n_points = 879
period = 0.5047530969540037
period_SNR = 106.63396025464299
period_log10FAP = -49.87638056245587
period_uncertainty = 9.194715699406553e-05
phase_cusum = 0.22247417418088664
phase_eta = 1.3317470074905482
phi21 = -2.704882110003022
phi31 = -1.862556647350471
quartile31 = 0.7072286713658968
r21 = 0.5079965438989301
r31 = 0.24121391743814052
shapiro_w = 0.9761942625045776
skewness = -0.3280184239631997
slope_per10 = -0.012078389638241632
slope_per90 = 0.011705945148283789
stetson_k = 0.6172692543542503
weighted_mean = 17.44664896945939
weighted_std = 0.44127232420929663
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 1.0


######### Lightcurve features #########

amplitude = 0.19424881679051437
cusum = 0.10203171570624923
eta = 2.1385363382664044
hl_amp_ratio = 0.34783602651267503
kurtosis = -0.425316126805952
n_points = 1345
period = 0.5946316313563619
period_SNR = 190.3815055463475
period_log10FAP = -154.61411390613216
period_uncertainty = 0.00012776786800899886
phase_cusum = 0.22778930507736841
phase_eta = 0.6449250545463119
phi21 = 0.6221185914306218
phi31 = 1.5312682374527329
quartile31 = 0.272544941719552
r21 = 0.4554864187272973
r31 = 0.21240426867259737
shapiro_w = 0.917834460735321
skewness = -0.7488185263220661
slope_per10 = -0.038623994788294834
slope_per90 = 0.03716606990882367
stetson_k = 0.690935119329278
weighted_mean = 14.789870649314969
weighted_std = 0.18273211269978998
